In [152]:
#Iportação das libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import missingno as msno
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [153]:
df = pd.read_csv("https://raw.githubusercontent.com/irvin-s/in-1166-smd/main/missao_2/data/crx_semantic.data", sep=',')

In [154]:
#Pré-processando os dados
df = df.replace('?', np.nan)
#Alterando os tipos das variáveis
df['Age'] = df['Age'].astype(np.float16)
#Atualizando valores nulos da variável Age com sua mediana. 
df.Age.fillna(df.Age.median(), inplace=True)
#Atualizando os valores nulos das variáveis Sex, Married, BankCostumer, EducationLevel, Ethnicity com os valores mais frequentes
a1_freq = df.Sex.value_counts().index[0]
a4_freq = df.Married.value_counts().index[0]
a5_freq = df.BankCostumer.value_counts().index[0]
a6_freq = df.EducationLevel.value_counts().index[0]
a7_freq = df.Ethnicity.value_counts().index[0]
df.Sex.fillna(a1_freq, inplace=True)
df.Married.fillna(a4_freq, inplace=True)
df.BankCostumer.fillna(a5_freq, inplace=True)
df.EducationLevel.fillna(a6_freq, inplace=True)
df.Ethnicity.fillna(a7_freq, inplace=True)
#Tranformar dados catégoricos em binários
df_dummified = pd.get_dummies(df, columns=['Sex', 'Married', 'BankCostumer', 'EducationLevel', 'Ethnicity', 'PriorDefault', 'Employed', 'DriversLicense', 'Citizen'])
df_dummified['Approved'] = df_dummified['Approved'].replace('+', 1)
df_dummified['Approved'] = df_dummified['Approved'].replace('-', 0)
#Normalizando os dados e criando Conjunto de Treino e Testes
X = df_dummified.drop(['Approved', 'ZipCode'], axis=1)
y = df_dummified['Approved']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)

## Normalizando os dados
scaler = MinMaxScaler()

scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Foram utilizadas várias téncias de pré-processamento, o dataset foi separado em treinamento e teste. As variáveis numéricas foram normalizadas para uma mesma escala.

In [161]:
#Minerando com KNN

#Definindo o valor de vizinhos
classifier = KNeighborsClassifier(n_neighbors=5)

#Treinar o modelo, com os dados de treinamento
classifier.fit(X_train, y_train)

#Prever os valores de y com dos dados de X_test
y_pred = classifier.predict(X_test)

# Imprimindo a matriz confusa
print("Matriz Confusa: ")
print(confusion_matrix(y_test, y_pred), "\n")  

# Imprimindo o relatório de classificação
print("Relatório de classificação: \n", classification_report(y_test, y_pred))  

# Imprimindo o quão acurado foi o modelo
print('Acurácia do modelo: ' , accuracy_score(y_test, y_pred))

Matriz Confusa: 
[[112   9]
 [ 18  89]] 

Relatório de classificação: 
               precision    recall  f1-score   support

           0       0.86      0.93      0.89       121
           1       0.91      0.83      0.87       107

    accuracy                           0.88       228
   macro avg       0.88      0.88      0.88       228
weighted avg       0.88      0.88      0.88       228

Acurácia do modelo:  0.881578947368421


O algoritmo KNN apresentou uma precisão de 86% para pedidos de crédito negados e de 91% para os pedidos aprovados. A acurácia do modelo foi de 88%. Para ter uma acurácia superior será necessário procurar outro valor de vizinhos(K).

In [ ]:
#Minerando com LVQ
